In [2]:
import pandas as pd
import xgboost as xgb

In [3]:
# train_df_path = "data/train.csv"
# test_df_path = "data/test.csv"
# train_df = pd.read_csv(train_df_path)
# test_df = pd.read_csv(test_df_path)
new_df_path = "/Users/user/Documents/Coding/cro_location_intelligence/src/data/all_data_embedding.csv"
df = pd.read_csv(new_df_path)
df.head()

,store_id,sum_pop_in_voronoi,voronoi_area,voronoi_density,cro_open_year,CRO_store_length,CRO_store_parking,CRO_store_stock_area,CRO_store_total_area,CRO_store_width,...,embedding_503,embedding_504,embedding_505,embedding_506,embedding_507,embedding_508,embedding_509,embedding_510,embedding_511,embedding_512
0,1025.0,16369.32,38412.2,0.426149,16.0,20.00,0.0,97.39,274.39,14.00,...,0.618719,0.397206,0.539024,0.034935,-0.425025,-0.202426,-0.603762,-0.113011,0.282425,-0.264580
1,8396.0,15736.69,241705.5,0.065107,16.0,15.35,0.0,44.86,175.86,19.80,...,0.853443,0.604886,0.487524,0.250037,-0.343431,0.215231,-0.492256,-0.528084,-0.124986,-0.288903
2,8572.0,16156.14,193980.1,0.083288,16.0,9.85,0.0,12.90,145.90,17.80,...,0.461060,0.396897,0.465704,0.372753,0.014367,0.028150,-0.261129,-0.084676,-0.178961,0.154477
3,1058.0,11213.96,77343.2,0.144990,16.0,13.94,0.0,11.83,93.83,7.90,...,0.755208,0.682671,0.673349,-0.079623,-0.562957,-0.830060,-0.801435,-0.366279,0.880094,0.253283
4,1095.0,16216.47,90980.3,0.178242,16.0,15.82,0.0,35.18,113.18,7.94,...,0.432766,0.767998,0.388810,0.492982,0.151838,-0.086584,-0.651078,-0.546258,-0.215056,-0.015719


In [4]:
# use embedding to predict mockup_sale with xgboost
from sklearn.model_selection import train_test_split
import xgboost as xgb
import numpy as np

# linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error

# get all column has embedding_ prefix
# feature_columns = [col for col in df.columns if col.startswith("embedding_")]
# feature_columns += ["subset"]
# feature_columns += ["y_nor"]
# assign to x
# all_data = df[feature_columns]

In [5]:
# normalize every column
from sklearn.preprocessing import StandardScaler

all_data = df.copy()
all_data = all_data.drop(columns=["store_id"])
all_data = all_data.drop(columns=["y_nor"])
train_all_data = all_data[all_data.subset == "train"]
all_data = all_data.drop(columns=["subset"])
train_all_data = train_all_data.drop(columns=["subset"])
scaler = StandardScaler()
scaler.fit(train_all_data)
all_data = scaler.transform(all_data)
all_data = pd.DataFrame(all_data)
# all_data.columns = feature_columns
# all_data["store_id"] = df["store_id"]
all_data["y_nor"] = df["y_nor"]
all_data["subset"] = df["subset"]
all_data.head()

,0,1,2,3,4,5,6,7,8,9,...,886,887,888,889,890,891,892,893,y_nor,subset
0,0.809053,-0.435626,1.276019,-0.585552,1.119992,-0.60246,1.958702,1.316572,-0.099400,1.694327,...,0.457054,-0.610891,-0.983325,-0.104843,-0.690702,0.676902,-0.050310,-0.970355,0.226238,train
1,0.587070,-0.246327,-0.271598,-0.585552,0.117934,-0.60246,-0.168104,-0.166140,0.836214,1.078724,...,0.214278,0.107851,-0.604111,1.191268,-0.142626,-0.891032,-1.833997,-1.052687,0.182798,train
2,0.734251,-0.290767,-0.193664,-0.585552,-1.067295,-0.60246,-1.462083,-0.616988,0.513588,-0.768087,...,0.111415,0.517898,1.058781,0.610704,0.993412,0.783935,-2.070301,0.448145,0.433002,train
3,-0.999912,-0.399375,0.070823,-0.585552,-0.185915,-0.60246,-1.505405,-1.400555,-1.083407,0.463120,...,1.090279,-0.993677,-1.624377,-2.052572,-1.662305,-0.279813,2.566342,0.782599,0.221966,train
4,0.755420,-0.386677,0.213359,-0.585552,0.219217,-0.60246,-0.560023,-1.109370,-1.076954,0.463120,...,-0.251072,0.919632,1.697689,0.254650,-0.923269,-0.959682,-2.228328,-0.127967,0.149573,train


In [6]:
# create noise data
def create_noise_data(data, times_sample, noise_level=0.1):
    new_data = data.copy()
    assert times_sample > 0 and isinstance(times_sample, int)
    for _ in range(times_sample):
        noise = np.random.normal(0, noise_level, data.shape)

        noise_data = data + noise
        new_data = pd.concat([new_data, noise_data])
    return new_data

In [7]:
X = all_data[all_data.subset == "train"]
X = X.drop(columns=["subset"])
y = X["y_nor"]
X_test = all_data[all_data.subset == "test"]
X_test = X_test.drop(columns=["subset"])
y_test = X_test["y_nor"]
X = X.drop(columns=["y_nor"])
X_test = X_test.drop(columns=["y_nor"])


# Assuming you have X, X_valid, y, y_valid
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.4, random_state=42
)
print(X_train.shape)
print(X_valid.shape)
X_train = create_noise_data(X_train, 10, noise_level=0.25)
# X_valid = create_noise_data(X_valid, 10, noise_level=0.25)
y_train = create_noise_data(y_train, 10, noise_level=0.1)
# y_valid = create_noise_data(y_valid, 10, noise_level=0.1)
print(X_train.shape)
print(X_valid.shape)

(586, 894)
(391, 894)
(6446, 894)
(391, 894)


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna
import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold

# Assuming train_df and test_df are your training and testing DataFrames

# Separate features (X) and target variable (y) in the training set
# X_train = train_df.drop(["y_nor", "store_id"], axis=1)
# y_train = train_df["y_nor"]

# # Separate features (X) and target variable (y) in the test set
# X_test = test_df.drop(["y_nor", "store_id"], axis=1)


def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "booster": "gbtree",
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "eta": trial.suggest_loguniform("eta", 1e-8, 1.0),
        "gamma": trial.suggest_loguniform("gamma", 1e-8, 1.0),
        "subsample": trial.suggest_uniform("subsample", 0.1, 1.0),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-4, 1.0),
    }

    model = xgb.XGBRegressor(**params, random_state=42)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        early_stopping_rounds=10,
        verbose=False,
    )
    y_pred = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)
    return rmse


# Create a study object and optimize the objective function using Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5, n_jobs=-1)

[I 2023-12-14 14:13:48,439] A new study created in memory with name: no-name-f052ec3c-2ea9-4675-b360-e3e95a21c034
/var/folders/9g/3rr3k99j0td2974k71sy46_h0000gp/T/ipykernel_62948/1599092795.py:23: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
/var/folders/9g/3rr3k99j0td2974k71sy46_h0000gp/T/ipykernel_62948/1599092795.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
/var/folders/9g/3rr3k99j0td2974k71sy46_h0000gp/T/ipykernel_62948/1599092795.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See ht

KeyboardInterrupt: 

In [9]:
study.best_value

0.08989768494458099

In [19]:
import xgboost as xgb

# Get the best hyperparameters from the study
best_params = study.best_params
print("Best Hyperparameters:", best_params)
display(best_params)
# Create the final model using the best hyperparameters
# final_model = ExtraTreesRegressor(**best_params, random_state=42)
# final_model = ExtraTreesRegressor()
final_model = xgb.XGBRegressor(**best_params, random_state=42)
# final_model = xgb.XGBRegressor()

# Train the final model on the entire training set
final_model.fit(X, y)

# Make predictions on the test set
y_pred_test = final_model.predict(X_test)

Best Hyperparameters: {'lambda': 0.8226516227759868, 'alpha': 1.4135427462827407e-05, 'max_depth': 6, 'eta': 6.5886252538597155e-06, 'gamma': 0.00020302042169817732, 'subsample': 0.510358433878495, 'colsample_bytree': 0.36795046116791585, 'min_child_weight': 7, 'n_estimators': 208, 'learning_rate': 0.102853525134955}


{'lambda': 0.8226516227759868,
 'alpha': 1.4135427462827407e-05,
 'max_depth': 6,
 'eta': 6.5886252538597155e-06,
 'gamma': 0.00020302042169817732,
 'subsample': 0.510358433878495,
 'colsample_bytree': 0.36795046116791585,
 'min_child_weight': 7,
 'n_estimators': 208,
 'learning_rate': 0.102853525134955}

[I 2023-12-14 14:19:57,028] Trial 4 finished with value: 0.11428656366908585 and parameters: {'lambda': 0.010256838647360881, 'alpha': 1.960632320172698e-06, 'max_depth': 16, 'eta': 0.00029130595379468923, 'gamma': 2.062293051506125e-06, 'subsample': 0.4750957701306353, 'colsample_bytree': 0.2938224515497027, 'min_child_weight': 2, 'n_estimators': 763, 'learning_rate': 0.00012685726665235208}. Best is trial 3 with value: 0.08989768494458099.


In [14]:
len(y_pred_test)

68

In [15]:
y_pred_test = final_model.predict(X_test)
y_pred_train = final_model.predict(X_train)
y_pred_valid = final_model.predict(X_valid)
test_df = X_test.copy()
test_df["y_nor"] = y_test
train_df = X_train.copy()
train_df["y_nor"] = y_train
val_df = X_valid.copy()
val_df["y_nor"] = y_valid

# Add the predictions to the test_df DataFrame
test_df["predicted_y_nor"] = y_pred_test

train_df["predicted_y_nor"] = y_pred_train
val_df["predicted_y_nor"] = y_pred_valid
# Map store_id back to the test_df DataFrame

In [16]:
from sklearn.metrics import mean_absolute_percentage_error

# Assuming you have the actual target values for the test set in y_test
y_test = test_df["y_nor"]

# Extract predicted values from the DataFrame
predictions = test_df["predicted_y_nor"]
test_df["mape"] = abs(test_df["y_nor"] - test_df["predicted_y_nor"]) / test_df["y_nor"]


selected_columns = ["y_nor", "predicted_y_nor", "mape"]
test_df[selected_columns].head()
# count mape<0.15
test_correct = test_df[test_df["mape"] < 0.15].shape[0]
test_df.shape[0]
print("test_correct", test_correct)
print(f"accuracy {test_correct / test_df.shape[0]:.2f}")

test_correct 21
accuracy 0.31


In [17]:
# val
val_df["mape"] = abs(val_df["y_nor"] - val_df["predicted_y_nor"]) / val_df["y_nor"]
val_correct = val_df[val_df["mape"] < 0.15].shape[0]
val_df.shape[0]
print("val_correct", val_correct)
print(f"accuracy {val_correct / val_df.shape[0]:.2f}")

val_correct 390
accuracy 1.00


In [18]:
# train
train_df["mape"] = (
    abs(train_df["y_nor"] - train_df["predicted_y_nor"]) / train_df["y_nor"]
)
train_correct = train_df[train_df["mape"] < 0.15].shape[0]
train_df.shape[0]
print("train_correct", train_correct)
print(f"accuracy {train_correct / train_df.shape[0]:.2f}")

train_correct 2219
accuracy 0.34
